In [165]:
import numpy as np
import sklearn as sk
import requests
from bs4 import BeautifulSoup
import pandas as pd
import sqlite3
from sqlite_api import *
from sklearn import linear_model

In [6]:
fdb = 'fantasy.db'

player_lbls = ['Player', 'Position', 'Team', 'Season', 'Plays', 'Games', 'RushAttempts', 
              'RushYrds', 'RushTDs', 'PassAttempts', 'Complete', 'PassYrds', 'PassTDs', 'Fumbles', 'Interceptions']

In [7]:
## Scoring metrics

qb = {'PassYrds': 0.04, # QBs get 0.04 points per passing yard 
        'PassTDs': 4, 
        'Interceptions': -2,
        'RushYrds': 0.1,
        'RushTDs': 6,
        'Fumbles': -2}

wr = {'PassYrds': 0.01,
         'PassTDs': 6,
         'Interceptions': -2,
         'RushYrds': 0.1,
         'RushTDs': 6,
         'Fumbles': -2}

rb = wr.copy()

te = wr.copy()

scores = {'QB': qb,
             'RB': rb,
             'WR': wr,
             'TE': te}

# need 2 points for 2point conversions

In [3]:
def score_row(row, qb_scores, wr_scores, rb_scores, te_scores):
    score = 0
    if row['Position'] == 'QB':
        metrics = qb_scores
    elif row['Position'] == 'WR':
        metrics = wr_scores
    elif row['Position'] == 'RB':
        metrics = rb_scores
    elif row['Position'] == 'TE':
        metrics = te_scores
    
    for var in metrics:
        score += row[var] * metrics[var]
        
    return score

In [12]:
select_to_df(fdb, 'OffenseSeason', ['Team', 'Season', 'Yards']).tail()

,Team,Season,Yards
91,SEA,2014,6414.0
92,SF,2014,5771.0
93,TB,2014,5215.0
94,TEN,2014,5159.0
95,WAS,2014,6177.0


In [6]:
plyr = select_to_df(fdb, 'PlayerSeason', player_lbls, where='WHERE Season=2016')  

In [8]:
# create 2016 mock season database
rdb = 'rankings.db'
conn = sqlite3.connect(rdb)
c = conn.cursor()

In [95]:
c.execute('DROP TABLE Kickers')

OperationalError: no such table: Kickers

In [96]:
for pos in ['Quarterbacks', 'RunningBacks', 'WideReceivers', 'TightEnds', 'Kickers']:
    c.execute('''CREATE Table ''' + pos +
                '''(Player VARCHAR(30),
                    Team VARCHAR(3),
                    Position CHARACTER(2),
                    PRIMARY KEY (Player))''')

In [97]:
c.execute('''CREATE Table Defenses
                (Team VARCHAR(3),
                Position CHARACTER(2),
                PRIMARY KEY (Team))''')

In [102]:
df = select_to_df(fdb, 'PlayerSeason', player_lbls, where='WHERE Season=2016')[['Player', 'Position', 'Team']]
QBs = df[df.Position == 'QB']
for idx in range(len(QBs)):
    row = df.iloc[idx]
    c.execute('INSERT INTO Quarterbacks VALUES (?, ?, ?)', (row[0], row[2], row[1]))

In [103]:
RBs = df[df.Position == 'RB']
for idx in range(len(RBs)):
    row = df.iloc[idx]
    c.execute('INSERT INTO RunningBacks VALUES (?, ?, ?)', (row[0], row[2], row[1]))

In [104]:
WRs = df[df.Position == 'WR']
for idx in range(len(WRs)):
    row = df.iloc[idx]
    c.execute('INSERT INTO WideReceivers VALUES (?, ?, ?)', (row[0], row[2], row[1]))

In [105]:
TEs = df[df.Position == 'TE']
for idx in range(len(TEs)):
    row = df.iloc[idx]
    c.execute('INSERT INTO TightEnds VALUES (?, ?, ?)', (row[0], row[2], row[1]))

In [107]:
df = select_to_df(fdb, 'KickerSeason', ['Player', 'Team', 'Position'], 'WHERE Season = 2016')
for idx in range(len(df)):
    c.execute('INSERT INTO Kickers VALUES (?, ?, ?)', df.iloc[idx])

In [108]:
df = select_to_df(fdb, 'DefenseSeason', ['Team', 'Position'], 'WHERE Season = 2016')
for idx in range(len(df)):
    c.execute('INSERT INTO Defenses VALUES (?, ?)', df.iloc[idx])

In [30]:
df = select_to_df(fdb, 'PlayerSeason', player_lbls, 'WHERE Season != 2016')
df['Score'] = ply.apply(score_row, args=(qb, wr, rb, te), axis=1)

In [64]:
for i in ply.Team.unique:
    print(i)

TypeError: 'method' object is not iterable

## Offensive Features

In [9]:
def add_feature(df, name, func):
    df[name] = df.apply(func, axis=1)
    return df

In [10]:
offense_lbls = ['Team', 
                 'Season', 
                 'Yards',
                 'PassYrds',
                 'RushYrds',
                 'Points']

In [11]:
def pass_rush_rat(row):
    return row['PassYrds'] / row['RushYrds']

def yards_per_game(row):
    return row['Yards'] / 16

def pass_yrds_per_game(row):
    return row['PassYrds'] / 16

def rush_yrds_per_game(row):
    return row['RushYrds'] / 16

def points_per_game(row):
    return row['Points'] / 16

In [12]:
off = select_to_df(fdb, 'OffenseSeason', offense_lbls)

In [13]:
off = add_feature(off, 'OffPassRushRatio', pass_rush_rat)
off = add_feature(off, 'OffYardsPerGame', yards_per_game)
off = add_feature(off, 'OffPassYrdsPerGame', pass_yrds_per_game)
off = add_feature(off, 'OffRushYrdsPerGame', rush_yrds_per_game)
off = add_feature(off, 'OffPointsPerGame', points_per_game)
off = off.drop(['Yards', 'PassYrds', 'RushYrds', 'Points'], axis=1)
off.head()

,Team,Season,OffPassRushRatio,OffYardsPerGame,OffPassYrdsPerGame,OffRushYrdsPerGame,OffPointsPerGame
0,NE,2007,2.558680,411.2500,295.6875,115.5625,36.8125
1,DAL,2007,2.351088,365.6875,256.5625,109.1250,28.4375
2,IND,2007,2.364009,358.6875,252.0625,106.6250,28.1250
3,JAX,2007,1.391886,357.4375,208.0000,149.4375,25.6875
4,SEA,2007,2.448425,348.9375,247.7500,101.1875,24.5625


## Additional Player Features

In [14]:
def pass_tdatt_rat(row):
    if row['PassAttempts'] == 0:
        return 0
    return row['PassTDs'] / row['PassAttempts']

def rush_tdatt_rat(row):
    if row['RushAttempts'] == 0:
        return 0
    return row['RushTDs'] / row['RushAttempts']

def pass_rush_att_rat(row):
    if row['RushAttempts'] == 0:
        return 1
    return row['PassAttempts'] / row['RushAttempts']

def complete_perc(row):
    if row['PassAttempts'] == 0:
        return 0
    return row['Complete'] / row['PassAttempts']

def avg_rush_yrds(row):
    if row['RushAttempts'] == 0:
        return 0
    return row['RushYrds'] / row['RushAttempts']

def avg_pass_yrds(row):
    if row['PassAttempts'] == 0:
        return 0
    return row['PassYrds'] / row['PassAttempts']

def avg_plays(row):
    return row['Plays'] / row['Games']

def score_per_play(row):
    return row['Score'] / row['Plays']

def score_per_game(row):
    return row['Score'] / row['Games']

def games_perc(row):
    return row['Games'] / 16

def intercept_rat(row):
    if row['PassAttempts'] == 0:
        return 0
    return row['Interceptions'] / row['PassAttempts']

def pass_att_per_game(row):
    return row['PassAttempts'] / row['Games']

def rush_att_per_game(row):
    return row['RushAttempts'] / row['Games']

def fumb_per_game(row):
    return row['Fumbles'] / row['Games']

In [15]:
df = select_to_df(fdb, 'PlayerSeason', player_lbls)
df['PassTDAttRatio'] = df.apply(pass_tdatt_rat, axis=1)
df['RushTDAttRatio'] = df.apply(rush_tdatt_rat, axis=1)
df['AvgPassYrds'] = df.apply(avg_pass_yrds, axis=1)
df['AvgRushYrds'] = df.apply(avg_rush_yrds, axis=1)
df['PercentComplete'] = df.apply(complete_perc, axis=1)
df['PlaysPerGame'] = df.apply(avg_plays, axis=1)
df['GamesPlayedPercent'] = df.apply(games_perc, axis=1)
df['InterceptionRatio'] = df.apply(intercept_rat, axis=1)
df['PassAttPerGame'] = df.apply(pass_att_per_game, axis=1)
df['RushAttPerGame'] = df.apply(rush_att_per_game, axis=1)
df['FumblesPerGame'] = df.apply(fumb_per_game, axis=1)
df['PassRushAttRatio'] = df.apply(pass_rush_att_rat, axis=1)
#df['ScorePerPlay'] = df.apply(score_per_play, axis=1)
#df['ScorePerGame'] = df.apply(score_per_game, axis=1)

In [16]:
players = pd.merge(df, off, on=['Team', 'Season'])
players.tail()

,Player,Position,Team,Season,Plays,Games,RushAttempts,RushYrds,RushTDs,PassAttempts,...,InterceptionRatio,PassAttPerGame,RushAttPerGame,FumblesPerGame,PassRushAttRatio,OffPassRushRatio,OffYardsPerGame,OffPassYrdsPerGame,OffRushYrdsPerGame,OffPointsPerGame
5224,Danny Vitale,RB,CLE,2016,5.0,9.0,0.0,0.0,0.0,5.0,...,0.0,0.555556,0.0,0.0,1.0,1.906542,311.0,204.0,107.0,16.5
5225,Gary Barnidge,TE,CLE,2016,82.0,16.0,0.0,0.0,0.0,82.0,...,0.0,5.125000,0.0,0.0,1.0,1.906542,311.0,204.0,107.0,16.5
5226,Seth DeValve,TE,CLE,2016,12.0,12.0,0.0,0.0,0.0,12.0,...,0.0,1.000000,0.0,0.0,1.0,1.906542,311.0,204.0,107.0,16.5
5227,Connor Hamlett,TE,CLE,2016,1.0,3.0,0.0,0.0,0.0,1.0,...,0.0,0.333333,0.0,0.0,1.0,1.906542,311.0,204.0,107.0,16.5
5228,Randall Telfer,TE,CLE,2016,7.0,14.0,0.0,0.0,0.0,7.0,...,0.0,0.500000,0.0,0.0,1.0,1.906542,311.0,204.0,107.0,16.5


## Creating the Models

In [310]:
class Model:
    def __init__(self, feature_list):
        self.features = feature_list
    
    def set_model(self, model):
        self.model = model
        
    def predict(self, feature_dict):
        parameters = list()
        for idx in range(len(self.features)):
            if self.features[idx][2:] in feature_dict:
                parameters.append(feature_dict[self.features[idx][2:]])
            else:
                parameters.append(feature_dict[self.features[idx]])
            
        return self.model.predict(parameters)
        
class PositionModels:
    def __init__(self, position):
        self.models = dict()
        self.position = position
    
    def add_model(self, predict, model):
        self.models[predict] = model
        
    def predict_player(self, feature_dict):
        predictions = dict()
        for cur_predict in self.models:
            if cur_predict[2:] in self.models:
                cur_model = self.models[cur_predict[2:]]
            else:
                cur_model = self.models[cur_predict]
            
            #print(str(cur_predict)+': '+str(cur_model.predict(feature_dict)))
            predictions[cur_predict] = cur_model.predict(feature_dict)
        
        return predictions
        
    def predict_df(self, df):
        def predict_var(row, to_predict):
            #print(len(self.predict_player(row.to_dict())[to_predict]))
            return self.predict_player(row.to_dict())[to_predict][0]
        
        for to_predict in self.models:
            print('Predicting '+to_predict+'...')
            new_col = 'Pred' + to_predict
            df[new_col] = df.apply(predict_var, args=(to_predict,), axis=1)
        
        return df

In [311]:
# todo: enable interactions
# get rid of zeros
# maybe get dont train on low games played
# maybe binary predict upward trend/downward trend

def create_model(df, pos, predict, features, reg_type='linear', game_limit=0, test_cutoff=2016):
    df = df[df.Position == pos]
    left_vars = ['Player', 'Team', 'Season'] + [predict]
    right_vars = ['Player', 'Team', 'Season'] + features
    model_vars = left_vars.copy()
    for idx in range(len(features)):
        features[idx] = 'PS' + features[idx]
    model_vars += features
        
    # build train dataframe
    model_df = pd.DataFrame(columns=model_vars)
    for season in range(2008, test_cutoff):
        prev_season = str(season - 1)
        season = str(season)
        left_df = df[df.GamesPlayedPercent >= (game_limit/16)]
        left_df = left_df[left_vars][df.Season == season]
        right_df = df[right_vars][df.Season == prev_season]
        right_df['Season'] = str(season)
        left_df = pd.merge(left_df, right_df, on=['Player', 'Team', 'Season'])
        left_df.columns = model_vars
        model_df = model_df.append(left_df)
        
    # build test dataframe
    test_df = pd.DataFrame(columns=model_vars)
    for season in range(test_cutoff, 2017):
        left_df = df[df.GamesPlayedPercent >= (game_limit/16)]
        left_df = left_df[left_vars][df.Season == str(season)]
        right_df = df[right_vars][df.Season == str(season-1)]
        right_df['Season'] = str(season)
        left_df = pd.merge(left_df, right_df, on=['Player', 'Team', 'Season'])
        left_df.columns = model_vars
        test_df = test_df.append(left_df)
  
    if reg_type == 'linear':
        model = linear_model.LinearRegression()
    elif reg_type == 'logistic':
        model = linear_model.LogisticRegression()
    model.fit(model_df[features], model_df[predict])
    print('|=== Predicting '+str(predict)+' ===|\n')
    print('|====== Model Coefficients ======|')
    for idx in range(len(model.coef_)):
        print(features[idx]+': '+str(model.coef_[idx]))
    print('Intercept: '+str(model.intercept_))
    print('\n|========== Performance ==========|')
    print('R-Sqr on Seasons '+str(test_cutoff)+' to 2016: '  + str(model.score(model_df[features], model_df[predict])))
    print('Train Data Size: '+str(len(model_df)))
    print('Test Data Size:'+str(len(test_df)))
    
    model_obj = Model(features)
    model_obj.set_model(model)
    return model_obj

### Quarterbacks

In [351]:
ModelsQB = PositionModels('QB')

In [369]:
AvgPassYrds = create_model(players, 'QB', 'AvgPassYrds', 
                           ['AvgPassYrds', 'PercentComplete', 'OffPassYrdsPerGame'], 
                   game_limit=10, test_cutoff=2014)
ModelsQB.add_model('AvgPassYrds', AvgPassYrds)

|=== Predicting AvgPassYrds ===|

|====== Model Coefficients ======|
PSAvgPassYrds: 0.281325343102
PSPercentComplete: 6.41163920165
PSOffPassYrdsPerGame: 0.000708867372633
Intercept: 1.13018502735

|========== Performance ==========|
R-Sqr on Seasons 2014 to 2016: 0.52100841563
Train Data Size: 143
Test Data Size:70


In [382]:
PassAttPerGame = create_model(players, 'QB', 'PassAttPerGame', 
                           ['PassAttPerGame', 'AvgPassYrds', 'PassTDAttRatio', 'OffPointsPerGame', 'GamesPlayedPercent'], 
                   game_limit=6, test_cutoff=2014)
ModelsQB.add_model('PassAttPerGame', PassAttPerGame)

|=== Predicting PassAttPerGame ===|

|====== Model Coefficients ======|
PSPassAttPerGame: 0.704405713448
PSAvgPassYrds: 0.731371374953
PSPassTDAttRatio: -31.0583377968
PSOffPointsPerGame: 0.309706790621
PSGamesPlayedPercent: -1.81155497123
Intercept: 0.26164063658

|========== Performance ==========|
R-Sqr on Seasons 2014 to 2016: 0.5826327795
Train Data Size: 184
Test Data Size:91


In [383]:
PassTDAttRatio = create_model(players, 'QB', 'PassTDAttRatio', 
                           ['PassTDAttRatio', 'AvgPassYrds', 'PercentComplete', 'GamesPlayedPercent', 'RushTDAttRatio'], 
                   game_limit=8, test_cutoff=2014)
ModelsQB.add_model('PassTDAttRatio', PassTDAttRatio)

|=== Predicting PassTDAttRatio ===|

|====== Model Coefficients ======|
PSPassTDAttRatio: 0.118074081227
PSAvgPassYrds: 0.00221885064841
PSPercentComplete: 0.0325126213073
PSGamesPlayedPercent: 0.0075644469316
PSRushTDAttRatio: 0.0508856621922
Intercept: -0.00569435143606

|========== Performance ==========|
R-Sqr on Seasons 2014 to 2016: 0.236048860829
Train Data Size: 161
Test Data Size:78


In [384]:
AvgRushYrds = create_model(players, 'QB', 'AvgRushYrds', 
                           ['AvgRushYrds', 'RushAttPerGame', 'PassAttPerGame', 'OffPointsPerGame'], 
                   game_limit=8, test_cutoff=2014)
ModelsQB.add_model('AvgRushYrds', AvgRushYrds)

|=== Predicting AvgRushYrds ===|

|====== Model Coefficients ======|
PSAvgRushYrds: 0.186097495049
PSRushAttPerGame: 0.606931980498
PSPassAttPerGame: -0.0429372088817
PSOffPointsPerGame: -0.0951528762998
Intercept: 4.5258439588

|========== Performance ==========|
R-Sqr on Seasons 2014 to 2016: 0.36818219927
Train Data Size: 161
Test Data Size:78


In [385]:
RushAttPerGame = create_model(players, 'QB', 'RushAttPerGame', 
                           ['RushAttPerGame', 'RushTDAttRatio', 'PassRushAttRatio', 'GamesPlayedPercent'], 
                   game_limit=6, test_cutoff=2014)
ModelsQB.add_model('RushAttPerGame', RushAttPerGame)

|=== Predicting RushAttPerGame ===|

|====== Model Coefficients ======|
PSRushAttPerGame: 0.637812988382
PSRushTDAttRatio: 4.63612815001
PSPassRushAttRatio: -0.0147349685176
PSGamesPlayedPercent: -0.330490929254
Intercept: 1.20301224329

|========== Performance ==========|
R-Sqr on Seasons 2014 to 2016: 0.495878521917
Train Data Size: 184
Test Data Size:91


In [386]:
FumblesPerGame = create_model(players, 'QB', 'FumblesPerGame', 
                           ['FumblesPerGame', 'OffPassYrdsPerGame', 'PassAttPerGame'], 
                   game_limit=8, test_cutoff=2014)
ModelsQB.add_model('FumblesPerGame', FumblesPerGame)

|=== Predicting FumblesPerGame ===|

|====== Model Coefficients ======|
PSFumblesPerGame: 0.0882807434402
PSOffPassYrdsPerGame: -0.000586994925773
PSPassAttPerGame: 0.00247949140662
Intercept: 0.260515590688

|========== Performance ==========|
R-Sqr on Seasons 2014 to 2016: 0.0696590095292
Train Data Size: 161
Test Data Size:78


In [387]:
playersQB = players[players.Position == 'QB']

In [388]:
PredictQB = ModelsQB.predict_df(playersQB)

Predicting FumblesPerGame...
Predicting PassAttPerGame...
Predicting RushAttPerGame...
Predicting AvgRushYrds...
Predicting AvgPassYrds...
Predicting PassTDAttRatio...


In [389]:
QB2015 = PredictQB[PredictQB.Season == '2015'][['Player', 'Season', 'Team', 'AvgPassYrds', 
            'PredAvgPassYrds', 'PassAttPerGame', 'PredPassAttPerGame']]
QB2016 = PredictQB[PredictQB.Season == '2016'][['Player', 'Season', 'Team', 'AvgPassYrds', 'PassAttPerGame']]
QB2016.columns = ['Player', 'Season', 'Team', '2016AvgPassYrds', '2016PassAttPerGame']
Predict2016 = pd.merge(QB2015, QB2016, on=('Player', 'Team'))
Predict2016 = Predict2016[['Player', 'Team', 'PredAvgPassYrds', '2016AvgPassYrds', 'PredPassAttPerGame', '2016PassAttPerGame']].sort(columns=('2016AvgPassYrds'), ascending=False)
Predict2016.head()

,Player,Team,PredAvgPassYrds,2016AvgPassYrds,PredPassAttPerGame,2016PassAttPerGame
22,Matt Ryan,ATL,7.677779,9.258427,36.449921,33.375000
15,Geno Smith,NYJ,7.206724,9.000000,40.360128,4.666667
1,Derek Anderson,CAR,7.251571,8.547170,15.397351,10.600000
21,Matt Moore,MIA,11.649222,8.287356,17.092592,17.400000
3,Tom Brady,NE,7.614500,8.226852,38.721706,36.000000


In [390]:
def predict_total_pass(row):
    return row.PredAvgPassYrds * row.PredPassAttPerGame * 16

def actual_total_pass(row):
    return row['2016AvgPassYrds'] * row['2016PassAttPerGame'] * 16

Predict2016['PredPassYrds'] = Predict2016.apply(predict_total_pass, axis=1)
Predict2016['2016PassYrds'] = Predict2016.apply(actual_total_pass, axis=1)
Predict2016 = Predict2016.sort(columns=('2016PassYrds'), ascending=False)
print('Total Correlation:')
print(Predict2016[['Player', 'PredPassYrds', '2016PassYrds']].corr())
print('Top 15 Correlation:')
print(Predict2016[['Player', 'PredPassYrds', '2016PassYrds']][:15].corr())
Predict2016[['Player', 'PredPassYrds', '2016PassYrds']]

Total Correlation:
              PredPassYrds  2016PassYrds
PredPassYrds      1.000000      0.694407
2016PassYrds      0.694407      1.000000
Top 15 Correlation:
              PredPassYrds  2016PassYrds
PredPassYrds      1.000000      0.571532
2016PassYrds      0.571532      1.000000


,Player,PredPassYrds,2016PassYrds
8,Drew Brees,5063.849120,5208.000000
22,Matt Ryan,4477.671080,4944.000000
11,Kirk Cousins,4299.119325,4917.000000
3,Tom Brady,4717.542660,4738.666667
33,Andrew Luck,4094.008997,4522.666667
6,Carson Palmer,4517.050167,4515.200000
16,Aaron Rodgers,3832.788392,4428.000000
12,Philip Rivers,4589.642609,4386.000000
28,Ben Roethlisberger,5075.675846,4364.571429
10,Matthew Stafford,4289.089819,4327.000000


In [391]:
Predict2016[['Player', 'PredPassYrds', '2016PassYrds']].sort(columns=('PredPassYrds'), ascending=False)

,Player,PredPassYrds,2016PassYrds
28,Ben Roethlisberger,5075.675846,4364.571429
8,Drew Brees,5063.849120,5208.000000
3,Tom Brady,4717.542660,4738.666667
15,Geno Smith,4653.828997,672.000000
12,Philip Rivers,4589.642609,4386.000000
32,Joe Flacco,4522.962637,4317.000000
6,Carson Palmer,4517.050167,4515.200000
22,Matt Ryan,4477.671080,4944.000000
9,Eli Manning,4388.220513,4027.000000
11,Kirk Cousins,4299.119325,4917.000000


In [37]:
ModelsQB.predict(players.iloc[3669].to_dict())

FumblesPerGame: [ 0.09054187]
PassAttPerGame: [ 10.49355906]
RushAttPerGame: [ 0.85778635]
AvgRushYrds: [ 2.05788415]
AvgPassYrds: [ 6.71199321]
PassTDAttRatio: [ 0.04019798]


C:\Users\Ryan\Anaconda3\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
C:\Users\Ryan\Anaconda3\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
C:\Users\Ryan\Anaconda3\lib\site-packages\sklearn\utils\validation.py:386: DeprecationWarning: Passing 1d arrays as data is deprecated in 0.17 and willraise ValueError in 0.19. Reshape your data either using X.reshape(-1, 1) if your data has a single feature or X.reshape(1, -1) if it contains a single sample.
  DeprecationWarning)
C

In [26]:
players[(players.Player == 'Benjamin Watson')].tail()

,Player,Position,Team,Season,Plays,Games,RushAttempts,RushYrds,RushTDs,PassAttempts,...,InterceptionRatio,PassAttPerGame,RushAttPerGame,FumblesPerGame,PassRushAttRatio,OffPassRushRatio,OffYardsPerGame,OffPassYrdsPerGame,OffRushYrdsPerGame,OffPointsPerGame
2412,Benjamin Watson,TE,CLE,2011,71.0,13.0,0.0,0.0,0.0,71.0,...,0.0,5.461538,0.0,0.0000,1.0,2.018289,288.8125,193.1250,95.6875,13.6250
2987,Benjamin Watson,TE,CLE,2012,82.0,16.0,0.0,0.0,0.0,82.0,...,0.0,5.125000,0.0,0.0000,1.0,2.156309,314.2500,214.6875,99.5625,18.8750
3130,Benjamin Watson,TE,NO,2013,30.0,15.0,0.0,0.0,0.0,30.0,...,0.0,2.000000,0.0,0.0000,1.0,3.338764,399.4375,307.3750,92.0625,25.8750
3669,Benjamin Watson,TE,NO,2014,31.0,16.0,0.0,0.0,0.0,31.0,...,0.0,1.937500,0.0,0.0000,1.0,2.620462,411.3750,297.7500,113.6250,25.0625
4239,Benjamin Watson,TE,NO,2015,110.0,16.0,0.0,0.0,0.0,110.0,...,0.0,6.875000,0.0,0.0625,1.0,3.333333,403.8125,310.6250,93.1875,25.5000


In [43]:
players[players.Position == 'QB']['PassAttPerGame'].std()

13.104008522968122

In [274]:
players.columns
# maybe add boolean columns is runner stuff like that

Index(['Player', 'Position', 'Team', 'Season', 'Plays', 'Games',
       'RushAttempts', 'RushYrds', 'RushTDs', 'PassAttempts', 'Complete',
       'PassYrds', 'PassTDs', 'Fumbles', 'Interceptions', 'PassTDAttRatio',
       'RushTDAttRatio', 'AvgPassYrds', 'AvgRushYrds', 'PercentComplete',
       'PlaysPerGame', 'GamesPlayedPercent', 'InterceptionRatio',
       'PassAttPerGame', 'RushAttPerGame', 'FumblesPerGame',
       'PassRushAttRatio', 'OffPassRushRatio', 'OffYardsPerGame',
       'OffPassYrdsPerGame', 'OffRushYrdsPerGame', 'OffPointsPerGame'],
      dtype='object')

### Running Backs